In [ ]:
# Install required packages from requirement.txt
import sys
!{sys.executable} -m pip install -r requirements.txt

     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ------------- ------------------------ 20.5/57.7 kB 330.3 kB/s eta 0:00:01
     -------------------- ----------------- 30.7/57.7 kB 330.3 kB/s eta 0:00:01
     --------------------------------- ---- 51.2/57.7 kB 375.8 kB/s eta 0:00:01
     -------------------------------------- 57.7/57.7 kB 338.8 kB/s eta 0:00:00
  Using cached typing_extensions-4.14.0-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/187.3 kB ? eta -:--:--
   ------------------- -------------------- 92.2/187.3 kB 2.6 MB/s eta 0:00:01
   -------------------------------- ------- 153.6/187.3 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 187.3/187.3 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/78.5 kB ? eta -:--:--
   ------------------------------------ --- 71.7/78.5 kB 2.0 MB/s eta 0:00:01
   


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: c:\Users\franc\Documents\GitHub\nlp_project\.venv\Scripts\python.exe -m pip install --upgrade pip


In [10]:
import os
import requests
from bs4 import BeautifulSoup
import csv
import time
import random
from tqdm import tqdm
from colorama import Fore, Style

In [5]:
# Constants
BASE_URL = "https://www.casarosada.gob.ar/informacion/discursos"
OUTPUT_DIR = "scraped_data"
CSV_FILE = os.path.join(OUTPUT_DIR, "speeches.csv")
PROGRESS_FILE = os.path.join(OUTPUT_DIR, "progress.txt")

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

In [6]:
# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)


def safe_get(url):
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        return response
    except requests.exceptions.RequestException as e:
        print(Fore.RED + f"[ERROR] No se pudo acceder a: {url}\n{e}" + Style.RESET_ALL)
        return None


def get_speech_links(page_url):
    response = safe_get(page_url)
    if not response:
        return []
    soup = BeautifulSoup(response.content, "html.parser")
    links = soup.find_all("a", class_="panel")
    return [link["href"] for link in links]


def scrape_speech(speech_url):
    response = safe_get(speech_url)
    if not response:
        return "No Title", "No Date", "No Content"

    soup = BeautifulSoup(response.content, "html.parser")

    title_tag = soup.find("h2", class_="panel-title")
    date_tag = soup.find("time")
    content_tag = soup.find("div", class_="item-page")

    title = title_tag.get_text(strip=True) if title_tag else "No Title"
    date = date_tag.get_text(strip=True) if date_tag else "No Date"
    content = content_tag.get_text(strip=True) if content_tag else "No Content"

    return title, date, content


def save_speech_to_txt(speech_id, date, content):
    formatted_date = date.replace(" ", "_").replace(",", "").replace(":", "")
    filename = os.path.join(OUTPUT_DIR, f"{speech_id}_{formatted_date}.txt")
    with open(filename, "w", encoding="utf-8") as file:
        file.write(content)


def load_progress():
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, "r") as file:
            return set(line.strip() for line in file)
    return set()


def save_progress(speech_id):
    with open(PROGRESS_FILE, "a") as file:
        file.write(f"{speech_id}\n")


def main():
    all_speeches = []
    page_number = 1
    speech_id = 1
    completed_speeches = load_progress()

    while True:
        page_url = f"{BASE_URL}?start={(page_number - 1) * 40}"
        print(Fore.GREEN + f"\nScraping página {page_number}..." + Style.RESET_ALL)
        speech_links = get_speech_links(page_url)

        if not speech_links:
            print(Fore.YELLOW + "No se encontraron más discursos. Fin del scraping." + Style.RESET_ALL)
            break

        for link in tqdm(speech_links, desc="Discursos en la página"):
            full_url = f"https://www.casarosada.gob.ar{link}"
            if str(speech_id) in completed_speeches:
                speech_id += 1
                continue

            title, date, content = scrape_speech(full_url)
            all_speeches.append([title, date, full_url, content])
            save_speech_to_txt(speech_id, date, content)
            save_progress(speech_id)
            speech_id += 1

            # Pausa aleatoria para evitar bloqueo
            time.sleep(random.uniform(1.5, 3.0))

        page_number += 1

    # Guardar CSV
    with open(CSV_FILE, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Title", "Date", "URL", "Content"])
        writer.writerows(all_speeches)

    print(Fore.BLUE + "\nScraping completado exitosamente." + Style.RESET_ALL)


if __name__ == "__main__":
    main()


Scraping página 1...


Discursos en la página:   2%|▎         | 1/40 [00:06<03:58,  6.11s/it]

[ERROR] No se pudo acceder a: https://www.casarosada.gob.ar/informacion/discursos/50997-palabras-del-presidente-de-la-nacion-javier-milei-luego-de-recibir-el-premio-escuela-de-salamanca-desde-el-centro-riojano-espana
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


Discursos en la página:  15%|█▌        | 6/40 [00:30<03:03,  5.39s/it]

[ERROR] No se pudo acceder a: https://www.casarosada.gob.ar/informacion/discursos/50992-palabras-del-presidente-de-la-nacion-javier-milei-en-jerusalen
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


...

[ERROR] No se pudo acceder a: https://www.casarosada.gob.ar/informacion/discursos/50612-palabras-del-presidente-de-la-nacion-en-congreso-de-inversiones-inmobiliarias
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


Discursos en la página:  85%|████████▌ | 34/40 [02:47<00:33,  5.53s/it]

[ERROR] No se pudo acceder a: https://www.casarosada.gob.ar/informacion/discursos/50593-palabras-del-presidente-de-la-nacion-javier-milei-en-la-exposicion-de-ganaderia-agricultura-e-industria-internacional-en-la-rural
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


Discursos en la página: 100%|██████████| 40/40 [03:18<00:00,  4.95s/it]



Scraping página 3...
[ERROR] No se pudo acceder a: https://www.casarosada.gob.ar/informacion/discursos?start=80
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
No se encontraron más discursos. Fin del scraping.

Scraping completado exitosamente.
